In [1]:
include("room.jl")
include("cell.jl")
#include("wall.jl")
include("climate.jl")
include("solar_radiation.jl")
include("wind_driven_rain.jl")
include("./function/lewis_relation.jl")

1.353437216624779e-7

## 1. 境界条件を表す構造体

### 1.1 第一種境界条件  
※BC:BoundaryCondition

In [2]:
Base.@kwdef mutable struct BC_Dirichlet
    name::String = "NoName"
    cell::Cell   = Cell()
end

BC_Dirichlet

In [ ]:
temp(state::BC_Dirichlet)= temp(state.cell)
miu(state::BC_Dirichlet) = miu(state.cell)
rh(state::BC_Dirichlet) = rh(state.cell)
pv(state::BC_Dirichlet)  = pv(state.cell)
ah(state::BC_Dirichlet)  = ah(state.cell)
phi(state::BC_Dirichlet)  = phi(state.cell)

### 1.2 第二種境界条件

In [3]:
Base.@kwdef mutable struct BC_Neumann
    name::String = "NoName"
    q::Float64  = 0.0
    jv::Float64 = 0.0
    jl::Float64 = 0.0
end

BC_Neumann

In [ ]:
temp(state::BC_Neumann)= 0.0
miu(state::BC_Neumann) = 0.0
rh(state::BC_Neumann) = 0.0
pv(state::BC_Neumann)  = 0.0
ah(state::BC_Neumann)  = 0.0
phi(state::BC_Neumann)  = 0.0

### 1.3 第三種境界条件

In [4]:
Base.@kwdef mutable struct BC_Robin
    name::String        = "NoName"
    air::Air            = Air()
    #wall::Cell          = Cell()    # 名称はcellの方が正しい
    cell::Cell          = Cell()
    q_added::Float64    = 0.0
    jv_added::Float64   = 0.0
    jl_added::Float64   = 0.0
    jl_surf::Float64    = 0.0 # 表面水（結露水）
    alpha::Float64      = 9.3
    alphac::Float64     = 4.9
    alphar::Float64     = 4.4
    aldm::Float64       = 3.2e-8
    θ::Dict{String, Float64}    = Dict( "a" => 0.0, # 方位角：azimuth
                                        "e" => 0.0, # 仰角　：elevation
                                        "LATIs" => -180.0,  # 日射が当たり始める方位　  ：LATI_started
                                        "LATIl" =>  360.0,  # 日射が当たる限界の方位　  ：LATI_limited
                                        "ALTIs" => -180.0,  # 日射が当たり始める高度    ：ALTI_started
                                        "ALTIl" =>  360.0)  # 日射が当たる限界の高度    ：ALTI_limited
    ar::Float64 = 0.0  # 日射吸収率（absorption rate）
    er::Float64 = 0.0  # 放射率（emissivity）
end

BC_Robin

#### 1.3.1 状態量の入出力

In [5]:
# 熱水分状態量
temp(state::BC_Robin)= state.air.temp
rh(state::BC_Robin) = state.air.rh
miu(state::BC_Robin) = convertRH2Miu( temp = temp(state.air), rh = rh(state.air) )
pv(state::BC_Robin)  = convertRH2Pv( temp = temp(state.air), rh = rh(state.air) )
ah(state::BC_Robin)  = convertPv2AH( patm = p_atm(state.air), pv = pv(state.air) )
patm(state::BC_Robin)= state.air.p_atm
vol(state::BC_Robin)= state.air.vol
material_name(state::BC_Dirichlet)= typeof(state)
material_name(state::BC_Neumann)= typeof(state)
material_name(state::BC_Robin)= typeof(state)

# 移動係数
alpha(state::BC_Robin) = state.alphac + state.alphar
aldm(state::BC_Robin)  = state.aldm
aldt(state::BC_Robin)  = aldm(state) * cal_DPvDT( temp = temp(state.air), miu = miu(state.air) )
aldmu(state::BC_Robin) = aldm(state) * cal_DPvDMiu( temp = temp(state.air), miu = miu(state.air) )
aldm_by_alphac(state::BC_Robin) = Lewis_relation.cal_aldm( alpha = state.alphac , temp = temp(state.air) )

θa(state::BC_Robin)     = state.θ["a"]
θe(state::BC_Robin)     = state.θ["e"]
LATI_started(state::BC_Robin)   = state.θ["LATIs"]
LATI_limited(state::BC_Robin)   = state.θ["LATIl"]
ALTI_started(state::BC_Robin)   = state.θ["ALTIs"]
ALTI_limited(state::BC_Robin)   = state.θ["ALTIl"]
ar(state::BC_Robin)     = state.ar
er(state::BC_Robin)     = state.er

# 値のセット
function set_air(state::BC_Robin, air::Air)
    state.air = air end
function set_cell(state::BC_Robin, cell::Cell)
    state.cell = cell end
function set_q_added(state::BC_Robin, q_added::Float64)
    state.q_added = q_added end
function set_jv_added(state::BC_Robin, jv_added::Float64)
    state.jv_added = jv_added end
function set_jl_added(state::BC_Robin, jl_added::Float64)
    state.jl_added = jl_added end
function set_jl_surf(state::BC_Robin, jl_surf::Float64)
    state.jl_surf = jl_surf end
function set_alpha(state::BC_Robin, alpha::Float64)
    state.alpha = alpha end
function set_alphac(state::BC_Robin, alphac::Float64)
    state.alphac = alphac end
function set_alphar(state::BC_Robin, alphar::Float64)
    state.alphar = alphar end
function set_aldm(state::BC_Robin, aldm::Float64)
    state.aldm = aldm end
function set_θa(state::BC_Robin, θa::Float64)
    state.θ["a"] = θa end
function set_θe(state::BC_Robin, θe::Float64)
    state.θ["e"] = θe end
function set_LATI_started(state::BC_Robin, LATI_started::Float64)
    state.θ["LATIs"] = LATI_started end
function set_LATI_limited(state::BC_Robin, LATI_limited::Float64)
    state.θ["LATIl"] = LATI_limited end
function set_ALTI_started(state::BC_Robin, ALTI_started::Float64)
    state.θ["ALTIs"] = ALTI_started end
function set_ALTI_limited(state::BC_Robin, ALTI_limited::Float64)
    state.θ["ALTIl"] = ALTI_limited end
function set_ar(state::BC_Robin, ar::Float64)
    state.ar = ar end
function set_er(state::BC_Robin, er::Float64)
    state.er = er end

# 値のセット
function add_q_added(state::BC_Robin, q_added::Float64)
    state.q_added = state.q_added + q_added end
function add_jv_added(state::BC_Robin, jv_added::Float64)
    state.jv_added = state.jv_added + jv_added end
function add_jl_added(state::BC_Robin, jl_added::Float64)
    state.jl_added = state.jl_added + jl_added end
function add_jl_surf(state::BC_Robin, jl_surf::Float64)
    state.jl_surf = state.jl_surf + jl_surf end

add_jl_surf (generic function with 1 method)

## 2. 日射量・降水量を境界条件（boundary condition）に入力する関数

### 2.1 壁面に入射する日射量の計算

In [6]:
# 直達日射量の計算
function cal_direct_solar_radiation_at_wall(climate::Climate, BC_Robin::BC_Robin)
    Jdi = cal_direct_solar_radiation_at_wall(
        SOLDN   = climate.solar["SOLDN"], 
        ALTI    = climate.solar["ALTI"], 
        LATI    = climate.solar["LATI"],
        θe      = θe(BC_Robin),
        θa      = θa(BC_Robin))
    return Jdi
end

cal_direct_solar_radiation_at_wall (generic function with 7 methods)

In [7]:
# 天空日射量の計算
function cal_diffused_solar_radiation_at_wall(climate::Climate, BC_Robin::BC_Robin)
    Jsi = cal_diffused_solar_radiation_at_wall(
        SOLSN   = climate.solar["SOLSN"],
        θe      = θe(BC_Robin))
    return Jsi
end

cal_diffused_solar_radiation_at_wall (generic function with 3 methods)

In [8]:
# 地面からの反射日射の計算  
function cal_reflected_solar_radiation_by_ground(climate::Climate, BC_Robin::BC_Robin)
    Jr  = cal_reflected_solar_radiation_by_ground( 
        SOLDN   = climate.solar["SOLDN"], 
        SOLSN   = climate.solar["SOLDN"],
        θe      = θe(BC_Robin),
        rho     = rho(climate) )
    return Jr
end

cal_reflected_solar_radiation_by_ground (generic function with 3 methods)

In [9]:
# 実効放射（夜間放射）の計算
function cal_effective_thermal_radiation_at_wall(climate::Climate, BC_Robin::BC_Robin)
    Jer = cal_effective_thermal_radiation_at_wall( 
        temp_wall   = temp(BC_Robin.cell),
        temp_air    = temp(climate.air), 
        ε           = er(BC_Robin), 
        pv          = pv(BC_Robin.air), 
        n           = cloudiness(climate),
        θe          = θe(BC_Robin))
    return Jer
end 

cal_effective_thermal_radiation_at_wall (generic function with 3 methods)

### 2.2 傾斜面に当たる降水量

傾斜面に当たる降水量$R{e}$は水平面降水量$R_h$と垂直面降水量$R_{wdr}$を用い以下のように表されるものとする。

$$
    R{e} = R_h \cos{\theta_e} + R_{wdr} \sin{\theta_e}
$$

また、降水量$R{e}$[mm/s]と境界面における水分流量$J_{l_{add}}$[kg/m2s]は以下の関係にある。

$$
    J_{l_{add}} = \rho_{w} \frac{R{e}}{1000}
$$

ただし、$\rho_{w}$は水の密度である。

In [10]:
# 壁面に当たる降水量の計算
function cal_effective_rainfall_at_wall(climate::Climate, BC_Robin::BC_Robin, α::Float64 = 0.222)
    Rh = climate.Jp / 3600.0    # 1sあたり降水量に変換
    Rwdr = cal_WDR_at_wall( Rh, WS(climate), θa(BC_Robin) - θwd(climate) , α )
    return Rh * cosd(θe(BC_Robin)) + Rwdr * sind(θe(BC_Robin))
end
cal_WDR_at_wall(; climate::Climate, BC_Robin::BC_Robin, α::Float64 = 0.222) = cal_WDR_at_wall(climate, BC_Robin, α)

cal_WDR_at_wall (generic function with 3 methods)

In [11]:
# 検証
include("wall.jl")
test_wall = Wall( ION = 30.0 )

# climate側の設定
test = input_climate_data( file_name = "../input_data/climate_data/sample_climate_data.csv", header = 3)
test_wall.BC_IP.air =  test.air
test.rho  = 0.1


0.1

In [12]:
# BCの構築
test_wall.cell[1].temp    = 280.15
test_wall.BC_IP.θ["e"] = 90.0
test_wall.BC_IP.θ["a"] = 270.0

test_wall.BC_IP.cell.temp =  280.15
test_wall.BC_IP.ar      =  0.9
test_wall.BC_IP.er      =  0.9

test.date = test.date + Day(18)
test.date = test.date + Hour(12)
for i = 1 : 2 * 24
    for I = [ :BC_IP, :BC_IM ]
        set_q_added( getfield(test_wall, I), 0.0)
    end
    
    reset_climate_data( test )
    Jdi = cal_direct_solar_radiation_at_wall(       test, test_wall.BC_IP   )
    Jsi = cal_diffused_solar_radiation_at_wall(     test, test_wall.BC_IP   )
    Jr  = cal_reflected_solar_radiation_by_ground(  test, test_wall.BC_IP   )
    Jer = cal_effective_thermal_radiation_at_wall(  test, test_wall.BC_IP   )
    Re  = cal_effective_rainfall_at_wall( test, test_wall.BC_IP )

    #println(test.date," , ", temp(test_wall.cell[1]) - 273.15," , ", temp(test.air) - 273.15," , ", Jer ) # + test.solar["SOLSN"] )
    #println(test.date," , ", Jdi," , ", Jsi," , ", Jr," , ", Jer," , ", test.solar["SOLDN"]," , ", test.solar["SOLSN"] ) # + test.solar["SOLSN"] )
    println(test.date," , WDR ", round(Re, digits = 5), ", 降水量 ",   round(test.Jp/3600.0, digits = 5), ", 風速 ",  WS(test) ) # + test.solar["SOLSN"] )
    test.date = test.date + Minute(30)
end

2012-01-19T12:00:00 , WDR 0.0, 降水量 0.0, 風速 3.8
2012-01-19T12:30:00 , WDR 2.0e-5, 降水量 7.0e-5, 風速 2.9
2012-01-19T13:00:00 , WDR -5.0e-5, 降水量 0.00014, 風速 2.0
2012-01-19T13:30:00 , WDR -0.00012, 降水量 0.00028, 風速 2.45
2012-01-19T14:00:00 , WDR -0.00021, 降水量 0.00042, 風速 2.9
2012-01-19T14:30:00 , WDR -0.00022, 降水量 0.00035, 風速 3.75
2012-01-19T15:00:00 , WDR 0.0001, 降水量 0.00028, 風速 4.6
2012-01-19T15:30:00 , WDR 0.00011, 降水量 0.00035, 風速 3.8
2012-01-19T16:00:00 , WDR 0.0001, 降水量 0.00042, 風速 3.0
2012-01-19T16:30:00 , WDR 0.00013, 降水量 0.00042, 風速 3.85
2012-01-19T17:00:00 , WDR 0.00016, 降水量 0.00042, 風速 4.7
2012-01-19T17:30:00 , WDR 0.00019, 降水量 0.00056, 風速 4.15
2012-01-19T18:00:00 , WDR 0.0002, 降水量 0.00069, 風速 3.6
2012-01-19T18:30:00 , WDR 0.00024, 降水量 0.00069, 風速 4.25
2012-01-19T19:00:00 , WDR 0.00028, 降水量 0.00069, 風速 4.9
2012-01-19T19:30:00 , WDR 0.00024, 降水量 0.00069, 風速 4.300000000000001
2012-01-19T20:00:00 , WDR 0.00021, 降水量 0.00069, 風速 3.7
2012-01-19T20:30:00 , WDR 0.00028, 降水量 0.00069, 風速 4.9
2

In [13]:
cal_effective_thermal_radiation_at_wall( 
        temp_wall   = 300.14, 
        temp_air    = 293.14, 
        ε           = 0.9, 
        pv          = 1000.0, 
        n           = 0, 
        θe          = 0.0)

-169.80698839945723

In [14]:
cal_effective_thermal_radiation_at_wall(  test, test_wall.BC_IP   )

-56.133817949290005